In [ ]:
#import plotting dependencies
import matplotlib
matplotlib.use('nbagg')
from matplotlib import style
style.use('seaborn')
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates


#import data analysis dependencies
import pandas as pd
import numpy as np

#import SQL Alchemy dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, desc

In [ ]:
#create engine
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
#prepare and reflect the database
Base = automap_base()
Base.prepare(engine, reflect=True)

In [ ]:
conn = engine.connect()
inspector = inspect(engine)

In [ ]:
#have a look at the keys
Base.classes.keys()

In [ ]:
#save each table as their own reference classes
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
session = Session(engine)

In [ ]:
## Start Precipitation Analysis 

In [ ]:
#query the database for a years worth of precipitation data. Focus on most recent.
precipitation_analysis_1 = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= "2016-08-23").\
    filter(Measurement.date <= "2017-08-23").all()

In [ ]:
#prec_df = pd.read_sql_query(precipitation_analysis_1, engine, index_col="date")

In [ ]:
#set index equal to date
pre_df = pd.DataFrame(precipitation_analysis_1)


In [ ]:
pre_df['date'] = pd.to_datetime(pre_df['date'])

In [ ]:
prec_df = pre_df.set_index('date')
prec_df.head()

In [ ]:
#plot the data
prec_plot = prec_df.plot(figsize = (16, 9), sort_columns=True, rot=45, use_index=True, legend=True, grid=True, color='b')
plt.ylabel('Precipitation', weight='bold')
plt.xlabel('Date', weight='bold')
plt.title("Precipitation in Hawaii from 8-23-2016 to 8-23-2017")
plt.show()

In [ ]:
#show short statistical analysis for data
prec_df.describe()

In [ ]:
## Start Station Analysis
station_num = session.query(Station.id).count()
station_num

In [ ]:
#Design a query to find the most active stations.
#List the stations and observation counts in descending order
#Which station has the highest number of observations?
most_active_station = session.query(Measurement.station, Station.name, func.count(Measurement.tobs)).\
filter(Measurement.station == Station.station).group_by(Measurement.station).order_by(func.count(Measurement.tobs).desc()).all()

In [ ]:
#Waihee 837.5 has the highest number of observations
most_active_station

In [ ]:
waihee_tobs = session.query(Measurement.tobs).\
filter(Measurement.station == "USC00519281", Measurement.station == Station.station, Measurement.date >="2016-08-23", Measurement.date <="2017-08-23").\
all()

In [ ]:
len(waihee_tobs)

In [ ]:
print(waihee_tobs[0][0])

In [ ]:
temperatures = [temp[0] for temp in waihee_tobs]

In [ ]:
plt.hist(temperatures, bins=12, color='b')
plt.title("Temperature Observation for Waihee Station (2016-08-23 to 2017)")
plt.ylabel('Frequency', weight='bold')
plt.xlabel('Temperature', weight='bold')
labels = ['tobs']
plt.legend(labels)

Write a function called calc_temps that will accept a start date and end date in the format %Y-%m-%d and return the minimum, average, and maximum temperatures for that range of dates.

Use the calc_temps function to calculate the min, avg, and max temperatures for your trip using the matching dates from the previous year (i.e. use "2017-01-01" if your trip start date was "2018-01-01")

Plot the min, avg, and max temperature from your previous query as a bar chart.

Use the average temperature as the bar height.

Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr).

In [ ]:
def calc_temps(start, end):
    query = session.query(Measurement.tobs).filter(Measurement.date>=start, Measurement.date<=end).all()
    temperatures = [temp[0] for temp in query]
    avg_temp = np.mean(temperatures)
    lowest_temp = min(temperatures)
    highest_temp = max(temperatures)
    

    plt.figure(figsize=(3,5))
    plt.bar(1, avg_temp, yerr=(highest_temp - lowest_temp), tick_label='', color='salmon')
    plt.ylabel("Temperature", weight="bold")
    plt.title("Average Trip Temperature", weight="bold")
    plt.show()

In [ ]:
columns = inspector.get_columns('Measurement')
for column in columns:
    print(column['name'])

In [ ]:
calc_temps(start="2017-08-07", end="2017-08-23")

In [ ]:
calc_temps(start="2013-08-12", end="2013-09-12")